<h3>Step 1: Importing neccacary Libararies

In [1]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import os
conn = sqlite3.connect('iotaDB.db')
import numpy as np

<h3> Step1: Read the CSV

In [2]:
# Load data
df = pd.read_csv('iota_tx_2024/iota_tx/IOTA_1year_tx_data2.csv', header=0)
print(df.columns)  # to check column names

Index(['transaction_id', 'block_index', 'input_addresses_x', 'input_amounts_x',
       'output_addresses_y', 'output_amounts_y', 'timestamp'],
      dtype='object')


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 942886 entries, 0 to 942885
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   transaction_id      942886 non-null  object 
 1   block_index         942886 non-null  float64
 2   input_addresses_x   942886 non-null  object 
 3   input_amounts_x     942886 non-null  object 
 4   output_addresses_y  942886 non-null  object 
 5   output_amounts_y    942886 non-null  object 
 6   timestamp           942886 non-null  object 
dtypes: float64(1), object(6)
memory usage: 50.4+ MB


 <h3>Step 2: Cleaning the Dataset</h3>



<h4>2.1 Identifying missing values, deleting the rows, saving in a new directory


<h5> 2.1.1 Identifying "not found" values in the dataset

In [4]:
def notfound_values(df):
    # Convert DataFrame to numpy array for fast operation
    data_array = df.values
    
    # Vectorized comparison over the numpy array
    is_not_found = (data_array == 'Not found')
    
    # Use numpy to check each row
    contains_not_found = np.any(is_not_found, axis=1)
    
    # Count the True values for rows containing 'Not found'
    not_found_count = np.sum(contains_not_found)
    
    print(f"Total rows with 'Not found': {not_found_count}")
    return not_found_count


notfound_count = notfound_values(df)

Total rows with 'Not found': 340


<h5> 2.1.2 Identifying "missing" values in the dataset

In [5]:
def missing_values (df):
  missing_values = df.isna().sum()
  print("Missing values in each column:")
  print(missing_values)
  return missing_values

missing_values_count = missing_values(df)

Missing values in each column:
transaction_id        0
block_index           0
input_addresses_x     0
input_amounts_x       0
output_addresses_y    0
output_amounts_y      0
timestamp             0
dtype: int64


<h5> 2.1.4 Cleaning the dataset

In [6]:

# Check for 'Not Found' across the entire DataFrame
mask = (df != 'Not found').all(axis=1)

# Count rows before filtering
initial_row_count = len(df)

# Apply the mask to filter out rows with 'Not Found'
cleaned_df = df[mask]

# Count rows after filtering
final_row_count = len(cleaned_df)
rows_deleted = initial_row_count - final_row_count

# Output the number of rows deleted
print(f"Total rows deleted: {rows_deleted}")




Total rows deleted: 340


In [7]:
# Drop all rows that have any missing values
df = df.dropna()

In [8]:
df = cleaned_df

<h4> 2.2 Adjusting the datatypes 

In [9]:

  # Convert timestamp to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

In [10]:
#Display the first 4 rows in the database
df.head()

,transaction_id,block_index,input_addresses_x,input_amounts_x,output_addresses_y,output_amounts_y,timestamp
0,65f4952557b7b021e65047c0666040ca81f0b56071b61c...,3033226.0,['704e8eccc32d7b314cc0cccbc29eaae59aa73c088016...,[500000000.0],['704e8eccc32d7b314cc0cccbc29eaae59aa73c088016...,"[25000000.0, 475000000.0]",2022-04-15 13:02:55
1,3fc9f4cfe97c60fe5ebd6b22d29c44874cac900eab4132...,3095432.0,['ef2e82cbdfb4cd7cff8c124deebe21774c860418b143...,"[60000000.0, 76000000.0]",['e319d445b5401cc40a3f86d739f1cf4faa3892891d22...,"[92000000.0, 44000000.0]",2022-04-22 17:50:35
2,db839f8d2511db74fa7f0bb8a630efb8c9f6c466aed5cc...,2646516.0,['8fe0e01639d3ee5ffdc9291ec2f7ba9b0500cee049ba...,[200000000.0],['22a98444cebb7b6938f5bb44bf3df90fcd2af3a7d2c7...,"[190000000.0, 10000000.0]",2022-03-01 18:25:43
3,a1d1953a998f11e0b2cf64e2f029b07d7813d1042bf9f3...,3033402.0,['8f26b244ee1228e0e9002bbde892599cc4e322da322e...,[25000000.0],['dea80a1c63c56f21beb18bae761b80d33fc0ad08ada0...,[25000000.0],2022-04-15 13:32:15
4,53b37aceaf50d0a7c73847cf7d5998d8068ffd19dfac8b...,3070044.0,['8ce2430b39c7a9e77e10ecd100da5fbf4549dfc55201...,[494500000.0],['8ce2430b39c7a9e77e10ecd100da5fbf4549dfc55201...,[494500000.0],2022-04-19 19:19:15


In [11]:
# Makeing sure about the right datatype
print(type(df['timestamp'].iloc[0]))
print(type(df['input_amounts_x'].iloc[0]))
print(type(df['input_addresses_x'].iloc[0]))
print(type(df['output_addresses_y'].iloc[0]))
print(type(df['output_amounts_y'].iloc[0]))
df.iloc[1]

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


transaction_id        3fc9f4cfe97c60fe5ebd6b22d29c44874cac900eab4132...
block_index                                                   3095432.0
input_addresses_x     ['ef2e82cbdfb4cd7cff8c124deebe21774c860418b143...
input_amounts_x                                [60000000.0, 76000000.0]
output_addresses_y    ['e319d445b5401cc40a3f86d739f1cf4faa3892891d22...
output_amounts_y                               [92000000.0, 44000000.0]
timestamp                                           2022-04-22 17:50:35
Name: 1, dtype: object

<h5> 2.1.3.1 Handling the datatype of input_adresses_x and output_addresses_y // Changing all the adresses with more than one input from strings to list

In [12]:
def parse_string_to_list(s):
    try:
        if s is None or isinstance(s, list):
            return []  # Handle None or already processed lists
        parsed_list = ast.literal_eval(s)

        if s is None or s == "Not found":
            print(f"Warning: Skipping due to None or Not found input at row : {s}")
            return []
        return parsed_list
    except (ValueError, SyntaxError, TypeError) as e:
        print(f"Error parsing or converting {s}: {e}")
        return []

In [13]:
df['input_addresses_x'] = df['input_addresses_x'].apply(parse_string_to_list)


In [14]:
df['output_addresses_y'] = df['output_addresses_y'].apply(parse_string_to_list)

In [15]:
print(type(df['input_addresses_x'].iloc[2]))
print(df['input_addresses_x'].iloc[1])

print(type(df['output_addresses_y'].iloc[2]))
print(df['output_addresses_y'].iloc[1])

<class 'list'>
['ef2e82cbdfb4cd7cff8c124deebe21774c860418b143ebbabe99e085833071c3', '1cfadd558641a3d501a4ea05e9ccc043d19180ac34ab197fff0b6671231c14ea']
<class 'list'>
['e319d445b5401cc40a3f86d739f1cf4faa3892891d228e8fd3f01598ff63f010', 'e90b9fc90d2429696b68da47c439f4853ffc4750580394b34273bb9c5510e65f']


<h5> 2.1.3.1 Handling the datatype of input_amount_x and output_amount_y

In [16]:
def parse_float_list(s):
    try:
        # Check for 'None' or the specific 'Not found' string
        if s is None or s == "Not found":
            print(f"Warning: Skipping due to None or Not found input at row : {s}")
            return []
        
        # Process the list if 's' is already a list
        if isinstance(s, list):
            return [float(item) for item in s]  # Convert each item to float
        
        # If 's' is a string, attempt to parse it as a literal list
        if isinstance(s, str):
            parsed_list = ast.literal_eval(s)
            return [float(item) for item in parsed_list]
        
        # Log any unexpected data types
        print(f"Unexpected data type : {type(s)} with value {s}")
        return []

    except (ValueError, SyntaxError, TypeError) as e:
        print(f"Error parsing or converting  {s}. Error: {e}")
        return []


In [17]:
df['output_amounts_y'] = df['output_amounts_y'].apply(parse_float_list)
df['input_amounts_x'] = df['input_amounts_x'].apply(parse_float_list)

In [18]:
# Makeing sure about the right datatype
print(type(df['input_amounts_x'].iloc[0]))
print(type(df['input_amounts_x'].iloc[0][0]))
print(type(df['input_addresses_x'].iloc[0]))

print(type(df['output_amounts_y'].iloc[0]))
print(type(df['output_amounts_y'].iloc[0][0]))
print(type(df['output_addresses_y'].iloc[0]))


<class 'list'>
<class 'float'>
<class 'list'>
<class 'list'>
<class 'float'>
<class 'list'>


In [20]:
# Makeing sure about the right datatype

print(type(df['input_addresses_x'].iloc[1]))
print(df['input_addresses_x'].iloc[1])

print(type(df['input_amounts_x'].iloc[1]))
print(df['input_amounts_x'].iloc[1])



print(type(df['output_addresses_y'].iloc[1]))
print(df['output_addresses_y'].iloc[1])

print(type(df['output_amounts_y'].iloc[1]))
print(df['output_amounts_y'].iloc[1])

print(type(df['output_amounts_y'].iloc[1][0]))
print(df['output_amounts_y'].iloc[1][0])




<class 'list'>
['ef2e82cbdfb4cd7cff8c124deebe21774c860418b143ebbabe99e085833071c3', '1cfadd558641a3d501a4ea05e9ccc043d19180ac34ab197fff0b6671231c14ea']
<class 'list'>
[60000000.0, 76000000.0]
<class 'list'>
['e319d445b5401cc40a3f86d739f1cf4faa3892891d228e8fd3f01598ff63f010', 'e90b9fc90d2429696b68da47c439f4853ffc4750580394b34273bb9c5510e65f']
<class 'list'>
[92000000.0, 44000000.0]
<class 'float'>
92000000.0


<h3>Step 4: Connecting to the database and inserting the cleaned and adjusted Dataframe</h3>

<h4> 4.1 Saving the list in json format to store in the Database (!List can not be stored in the database)

In [1]:
import json

for col in df.columns:
  if df[col].apply(lambda x: isinstance (x,list)).any():
    df[col] = df[col].apply(json.dumps)

print("DataFrame Structure:")
print(df.dtypes)

NameError: name 'df' is not defined

<h4> 4.2 Creating the Database and createing the Table for the Datasets

In [1]:
# Create a table with corrected column names and data types
conn = sqlite3.connect('iotaDB.db')
c = conn.cursor()

c.execute('''
CREATE TABLE Transactions (
    transaction_id ,
    block_index ,
    input_addresses_x ,
    input_amounts_x ,
    output_addresses_y ,
    output_amounts_y ,
    timestamp 
)
''')

# Use a default single insert statement per row
df.to_sql('Transactions', conn, if_exists='replace', index=False, method=None)


# Commit changes and close the connection
conn.commit()
conn.close()





NameError: name 'sqlite3' is not defined

Working with JSON strings in the Data

Verification of List in the Database

In [25]:
import json
import sqlite3

# Example database fetch code
conn = sqlite3.connect('iotaDB.db')
c = conn.cursor()

# Query to fetch serialized data
c.execute("""
SELECT input_addresses_x, output_addresses_y, input_amounts_x, output_amounts_y 
FROM Transactions 
LIMIT 10
""")
rows = c.fetchall()

# Deserialize the JSON string back into Python lists for all four columns
addresses_and_amounts = [{
    'input_addresses': json.loads(row[0]) if row[0] else None,
    'output_addresses': json.loads(row[1]) if row[1] else None,
    'input_amounts': json.loads(row[2]) if row[2] else None,
    'output_amounts': json.loads(row[3]) if row[3] else None
} for row in rows]

# Output the deserialized data along with types
for idx, item in enumerate(addresses_and_amounts, start=1):
    print(f"Record {idx} - Input Addresses: {item['input_addresses']} (type: {type(item['input_addresses'])})")
    print(f"Record {idx} - Output Addresses: {item['output_addresses']} (type: {type(item['output_addresses'])})")
    print(f"Record {idx} - Input Amounts: {item['input_amounts']} (type: {type(item['input_amounts'])})")
    print(f"Record {idx} - Output Amounts: {item['output_amounts']} (type: {type(item['output_amounts'])})")

# Close the database connection


Record 1 - Input Addresses: ['704e8eccc32d7b314cc0cccbc29eaae59aa73c0880161d52b9b87c13d4703896'] (type: <class 'list'>)
Record 1 - Output Addresses: ['704e8eccc32d7b314cc0cccbc29eaae59aa73c0880161d52b9b87c13d4703896', 'dea80a1c63c56f21beb18bae761b80d33fc0ad08ada018eb982f1dd313109aed'] (type: <class 'list'>)
Record 1 - Input Amounts: [500000000.0] (type: <class 'list'>)
Record 1 - Output Amounts: [25000000.0, 475000000.0] (type: <class 'list'>)
Record 2 - Input Addresses: ['ef2e82cbdfb4cd7cff8c124deebe21774c860418b143ebbabe99e085833071c3', '1cfadd558641a3d501a4ea05e9ccc043d19180ac34ab197fff0b6671231c14ea'] (type: <class 'list'>)
Record 2 - Output Addresses: ['e319d445b5401cc40a3f86d739f1cf4faa3892891d228e8fd3f01598ff63f010', 'e90b9fc90d2429696b68da47c439f4853ffc4750580394b34273bb9c5510e65f'] (type: <class 'list'>)
Record 2 - Input Amounts: [60000000.0, 76000000.0] (type: <class 'list'>)
Record 2 - Output Amounts: [92000000.0, 44000000.0] (type: <class 'list'>)
Record 3 - Input Addresses

In [26]:
import json
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('iotaDB.db')
c = conn.cursor()

# Execute query to fetch serialized data
c.execute("""
SELECT input_amounts_x 
FROM Transactions 
LIMIT 10
""")
rows = c.fetchall()

# Deserialize the JSON string back into Python lists
input_amounts = [json.loads(row[0]) if row[0] else None for row in rows]

# Close the database connection
conn.close()


# Check and print the type of each element in the lists

In [27]:
# Check and print the type of each element in the lists
for idx, amounts in enumerate(input_amounts, start=1):
    if amounts:
        print(f"Record {idx} - Input Amounts Types:")
        for i, amount in enumerate(amounts):
            print(f"  Element {i+1}: {amount} (type: {type(amount)})")
    else:
        print(f"Record {idx} - No input amounts available.")


Record 1 - Input Amounts Types:
  Element 1: 500000000.0 (type: <class 'float'>)
Record 2 - Input Amounts Types:
  Element 1: 60000000.0 (type: <class 'float'>)
  Element 2: 76000000.0 (type: <class 'float'>)
Record 3 - Input Amounts Types:
  Element 1: 200000000.0 (type: <class 'float'>)
Record 4 - Input Amounts Types:
  Element 1: 25000000.0 (type: <class 'float'>)
Record 5 - Input Amounts Types:
  Element 1: 494500000.0 (type: <class 'float'>)
Record 6 - Input Amounts Types:
  Element 1: 29891752444.0 (type: <class 'float'>)
Record 7 - Input Amounts Types:
  Element 1: 22994863356.0 (type: <class 'float'>)
Record 8 - Input Amounts Types:
  Element 1: 1000000.0 (type: <class 'float'>)
Record 9 - Input Amounts Types:
  Element 1: 4501456962.0 (type: <class 'float'>)
Record 10 - Input Amounts Types:
  Element 1: 0.0 (type: <class 'float'>)
